In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [ ]:
data_train = pd.read_csv('Training_clean5.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5
data_test = pd.read_csv('Test_clean3.csv')

In [ ]:
#data_train['floor/stories'] = data_train.floor/data_train.stories
#data_test['floor/stories'] = data_test.floor/data_test.stories

data_train['distance'] = np.sqrt((37.6 - data_train['longitude'])**2 + (55.75 - data_train['latitude'])**2)
data_test['distance'] = np.sqrt((37.6 - data_test['longitude'])**2 + (55.75 - data_test['latitude'])**2)

#data_train.drop(['floor', 'stories', 'longitude', 'latitude'], axis=1, inplace=True)
data_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
#data_test.drop(['floor', 'stories', 'longitude', 'latitude'], axis=1, inplace=True)
data_test.drop(['longitude', 'latitude'], axis=1, inplace=True)
data_train.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,street,distance
0,0,3,7139520.0,59.2,12.5,31.0,2,2,1,3,...,3,9,0,1,1,1,1,0,Бунинские Луга ЖК,0.239348
1,1,0,10500000.0,88.0,14.2,48.0,18,3,1,3,...,3,25,0,1,1,1,1,0,улица Радужная,0.129713
2,2,3,9019650.0,78.5,22.5,40.8,12,3,1,3,...,3,15,0,1,1,1,0,0,Саларьево Парк ЖК,0.121183
3,3,0,10500000.0,88.0,14.0,48.0,18,3,1,3,...,3,25,0,1,1,1,1,0,улица Радужная,0.129713
4,4,1,13900000.0,78.0,17.0,35.0,7,2,1,3,...,2,15,0,1,1,1,0,0,улица Сервантеса,0.217762


In [ ]:
target_train = data_train.price/data_train.area_total
y_train_area_total = data_train.area_total
y_test_area_total = data_test.area_total
data_train.drop(['price', 'area_total'], axis=1, inplace=True)
data_train.head()
data_test.drop(['area_total'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
data_train['street'] = ord_enc.fit_transform(data_train[["street"]]).astype(int)
data_train.head()
data_test['street'] = ord_enc.fit_transform(data_test[["street"]]).astype(int)

In [ ]:
X_train1, X_val1, y_train, y_val = train_test_split(data_train, target_train, test_size = 0.2, random_state=42)

X_train1.shape, X_val1.shape, y_train.shape, y_val.shape
X_train = X_train1.drop(['id'], axis=1)
X_val = X_val1.drop(['id'], axis=1)

In [ ]:
model = RandomForestRegressor(n_jobs= -1, random_state = 0, bootstrap=True)

In [ ]:
param_grid =  {'n_estimators':[200, 300, 400, 500, 600],
               'max_features':[0.1, 0.3, 0.6]
              }

In [ ]:
Tuned_RandForest = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5)

In [ ]:
Tuned_RandForest.fit(data_train.drop(['id'], axis=1), y=target_train)

KeyboardInterrupt: 

In [ ]:
results = pd.DataFrame(Tuned_RandForest.cv_results_)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>